In [ ]:
python -m pip install -q awscli
export AWS_EC2_METADATA_DISABLED=true

mkdir -p data/train data/val

aws s3 cp --no-sign-request s3://cd2688-object-detection-tf2/train/ data/train --recursive
aws s3 cp --no-sign-request s3://cd2688-object-detection-tf2/val/   data/val   --recursive

ls data/train | head
ls data/val | head



In [ ]:
cat > data/label_map.pbtxt << 'EOF'
item { id: 1 name: 'vehicle' }
item { id: 2 name: 'pedestrian' }
item { id: 4 name: 'cyclist' }
EOF

cat data/label_map.pbtxt


In [ ]:
mkdir -p pretrained_models
cd pretrained_models

wget -nc http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
tar -xzf ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

wget -nc http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
tar -xzf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

cd ..


In [ ]:
mkdir -p configs

cp pretrained_models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config configs/ssd_pipeline.config
cp pretrained_models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/pipeline.config configs/frcnn_pipeline.config


In [ ]:
python - <<'PY'
import re, pathlib

PROJ = pathlib.Path(".").resolve()

def patch_cfg(cfg_path, ckpt_dir, batch):
    p = pathlib.Path(cfg_path)
    txt = p.read_text()

    # num_classes
    txt = re.sub(r"num_classes:\s*\d+", "num_classes: 3", txt)

    # batch size (first occurrence, inside train_config)
    txt = re.sub(r"batch_size:\s*\d+", f"batch_size: {batch}", txt, count=1)

    # checkpoint path to ckpt-0
    ckpt0 = (PROJ / ckpt_dir / "checkpoint" / "ckpt-0").as_posix()
    txt = re.sub(r'fine_tune_checkpoint:\s*".*?"', f'fine_tune_checkpoint: "{ckpt0}"', txt)
    txt = re.sub(r'fine_tune_checkpoint_type:\s*".*?"', 'fine_tune_checkpoint_type: "detection"', txt)

    # label map
    lmap = (PROJ / "data" / "label_map.pbtxt").as_posix()
    txt = re.sub(r'label_map_path:\s*".*?"', f'label_map_path: "{lmap}"', txt)

    # TFRecords
    train_pat = (PROJ / "data" / "train" / "*.tfrecord").as_posix()
    val_pat   = (PROJ / "data" / "val"   / "*.tfrecord").as_posix()

    # Replace first input_path (train) then second (eval)
    txt = re.sub(r'input_path:\s*".*?"', f'input_path: "{train_pat}"', txt, count=1)
    txt = re.sub(r'input_path:\s*".*?"', f'input_path: "{val_pat}"', txt, count=1)

    p.write_text(txt)
    print(f"Patched: {p}")

patch_cfg(
    "configs/ssd_pipeline.config",
    "pretrained_models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8",
    batch=16
)

patch_cfg(
    "configs/frcnn_pipeline.config",
    "pretrained_models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8",
    batch=2
)
PY


In [ ]:
grep -n "num_classes" configs/ssd_pipeline.config | head
grep -n "num_classes" configs/frcnn_pipeline.config | head
grep -n "label_map_path" configs/ssd_pipeline.config | head
grep -n "input_path" configs/ssd_pipeline.config | head -n 5


In [ ]:
conda activate udacity-tfod
python -m pip install tf_slim lvis pycocotools


In [ ]:
cd ~/Object-Detection-in-Urban-Environments/cd2688-object-detection-in-urban-environment-project


In [ ]:
python - <<'PY'
import re, pathlib

PROJ = pathlib.Path(".").resolve()
cfg = PROJ / "configs/ssd_pipeline.config"
txt = cfg.read_text()

train_pat = (PROJ / "data" / "train" / "*.tfrecord").as_posix()
val_pat   = (PROJ / "data" / "val"   / "*.tfrecord").as_posix()

# Fix any placeholders
txt = txt.replace("PATH_TO_BE_CONFIGURED", train_pat)

# More robust: patch train_input_reader and eval_input_reader blocks
# Replace input_path inside train_input_reader block
txt = re.sub(
    r"(train_input_reader\s*\{[\s\S]*?input_path:\s*\")([^\"]*)(\"[\s\S]*?\})",
    lambda m: m.group(1) + train_pat + m.group(3),
    txt,
    count=1
)

# Replace input_path inside eval_input_reader block
txt = re.sub(
    r"(eval_input_reader\s*\{[\s\S]*?input_path:\s*\")([^\"]*)(\"[\s\S]*?\})",
    lambda m: m.group(1) + val_pat + m.group(3),
    txt,
    count=1
)

cfg.write_text(txt)
print("Patched:", cfg)
PY


In [ ]:
grep -n "train_input_reader" -n configs/ssd_pipeline.config | head -n 5
grep -n "eval_input_reader"  -n configs/ssd_pipeline.config | head -n 5
grep -n "input_path" configs/ssd_pipeline.config | head -n 10


In [ ]:
python - <<'PY'
import re, pathlib

PROJ = pathlib.Path(".").resolve()
cfg = PROJ / "configs/frcnn_pipeline.config"
txt = cfg.read_text()

train_pat = (PROJ / "data" / "train" / "*.tfrecord").as_posix()
val_pat   = (PROJ / "data" / "val"   / "*.tfrecord").as_posix()

txt = txt.replace("PATH_TO_BE_CONFIGURED", train_pat)

txt = re.sub(
    r"(train_input_reader\s*\{[\s\S]*?input_path:\s*\")([^\"]*)(\"[\s\S]*?\})",
    lambda m: m.group(1) + train_pat + m.group(3),
    txt,
    count=1
)

txt = re.sub(
    r"(eval_input_reader\s*\{[\s\S]*?input_path:\s*\")([^\"]*)(\"[\s\S]*?\})",
    lambda m: m.group(1) + val_pat + m.group(3),
    txt,
    count=1
)

cfg.write_text(txt)
print("Patched:", cfg)
PY



In [ ]:
grep -n "input_path" configs/frcnn_pipeline.config | head -n 10


In [ ]:
conda activate udacity-tfod

python -m pip uninstall -y numpy
python -m pip install "numpy==1.24.3"


In [ ]:
python - <<'PY'
import numpy as np
import tensorflow as tf
print("NumPy:", np.__version__)
print("TF:", tf.__version__)
PY


In [ ]:
cd ~/Object-Detection-in-Urban-Environments/cd2688-object-detection-in-urban-environment-project

cat > constraints.txt << 'EOF'
numpy==1.24.3
EOF


In [ ]:
python -m pip install -c constraints.txt <package>


In [ ]:
python -m pip install tf_slim


In [ ]:
conda activate udacity-tfod
python -m pip install "scipy==1.10.1"


In [ ]:
python -c "import numpy as np, scipy; print(np.__version__, scipy.__version__)"


In [ ]:
python -m pip install \
  pillow==10.4.0 \
  matplotlib==3.7.5 \
  pandas==2.0.3 \
  opencv-python==4.10.0.84 \
  protobuf==3.20.3 \
  lxml==4.9.4 \
  Cython==3.0.10 \
  contextlib2==21.6.0 \
  pycocotools==2.0.7


In [ ]:
grep -n "input_path" configs/ssd_pipeline.config | head -n 12


In [ ]:
conda activate udacity-tfod
python -m pip install "tensorflow-io==0.37.1"


In [ ]:
python -c "import tensorflow_io as tfio; print('tfio:', tfio.__version__)"


In [ ]:
conda activate udacity-tfod
conda install -y -c conda-forge "pyyaml=5.4.1"


In [ ]:
python -m pip install "Cython<3"


In [ ]:
python -m pip install --no-build-isolation --no-deps tf-models-official==2.13.0


In [ ]:
python -m pip install \
  gin-config==0.5.0 \
  tensorflow-addons==0.21.0 \
  tensorflow-datasets==4.9.2 \
  immutabledict==4.2.1 \
  seqeval==1.2.2 \
  sentencepiece==0.2.0 \
  tf-keras==2.13.1


In [ ]:
conda activate udacity-tfod
python -m pip install gin-config==0.5.0


In [ ]:
conda activate udacity-tfod
python -m pip install gin-config==0.5.0


In [ ]:
python - <<'PY'
import gin
print("gin OK:", gin.__version__)
PY


In [ ]:
conda activate udacity-tfod
python -m pip install "tensorflow-datasets==4.9.2"


In [ ]:
python -m pip install "tensorflow-metadata==1.13.0" "etils==1.7.0"


In [ ]:
conda activate udacity-tfod
python -m pip install immutabledict==4.2.1


In [ ]:
python - <<'PY'
import immutabledict
print("immutabledict OK:", immutabledict.__version__)
PY


In [ ]:
python - <<'PY'
from official.vision.image_classification.efficientnet import efficientnet_model
print("official EfficientNet import OK")
PY


In [ ]:
conda activate udacity-tfod

python - <<'PY'
import tensorflow as tf
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))
PY


In [ ]:
ls -l /usr/lib/wsl/lib/libcuda.so* || true
ls -l /usr/lib/x86_64-linux-gnu/libcuda.so* || true
nvidia-smi || true


In [ ]:
cd ~/Object-Detection-in-Urban-Environments/cd2688-object-detection-in-urban-environment-project
grep -n "batch_size" -n configs/ssd_pipeline.config | head -n 20


batch_size: 16 -> batch_size: 4

In [ ]:
fixed_shape_resizer {
        height: 640
        width: 640
      }
->

fixed_shape_resizer {
        height: 
        width: 
      }

In [ ]:
conda activate udacity-tfod
cd ~/Object-Detection-in-Urban-Environments/cd2688-object-detection-in-urban-environment-project
mkdir -p training/ssd

python ~/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path=configs/ssd_pipeline.config \
  --model_dir=training/ssd \
  --checkpoint_every_n=50 \
  --alsologtostderr


In [ ]:
export LD_LIBRARY_PATH=$(python - <<'PY'
import site,glob,os
sp=site.getsitepackages()[0]
paths=set()
for p in glob.glob(os.path.join(sp,"nvidia","**","lib"), recursive=True):
    paths.add(p)
for p in glob.glob(os.path.join(sp,"nvidia","**","lib64"), recursive=True):
    paths.add(p)
print(":".join(sorted(paths)))
PY
):$LD_LIBRARY_PATH


In [ ]:
cd ~/Object-Detection-in-Urban-Environments/cd2688-object-detection-in-urban-environment-project

mkdir -p training/ssd

python ~/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path=configs/ssd_pipeline.config \
  --model_dir=training/ssd \
  --checkpoint_every_n=200 \
  --alsologtostderr


In [ ]:
mkdir -p training/ssd

python ~/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path=configs/ssd_pipeline.config \
  --model_dir=training/ssd \
  --alsologtostderr
